# Fine Tune NER Model

#### Set Up Colab Environment

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_SILENT"] = "true"


In [ ]:
!pip install transformers datasets seqeval

####  Upload and Parse CoNLL File

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd

def load_conll(filepath):
    tokens, labels, all_tokens, all_labels = [], [], [], []

    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    all_tokens.append(tokens)
                    all_labels.append(labels)
                    tokens, labels = [], []
            else:
                token, label = line.split()
                tokens.append(token)
                labels.append(label)
        if tokens:
            all_tokens.append(tokens)
            all_labels.append(labels)
    return pd.DataFrame({"tokens": all_tokens, "ner_tags": all_labels})

df = load_conll('/content/labeled_data.conll')
dataset = Dataset.from_pandas(df)
dataset = DatasetDict({"train": dataset})
dataset


DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3498
    })
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Label Encoding

In [ ]:
unique_labels = list(set(label for sublist in df['ner_tags'] for label in sublist))
label2id = {label: i for i, label in enumerate(sorted(unique_labels))}
id2label = {i: label for label, i in label2id.items()}


#### Load Pre-trained best Tokenizer and Model (xlm-roberta-base)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Tokenize and Align Labels

In [ ]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(
        example["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding='max_length',
        max_length=128
    )

    word_ids = tokenized_inputs.word_ids()
    label_ids = []
    previous_word_idx = None

    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(label2id[example["ner_tags"][word_idx]])
        else:
            label_ids.append(label2id[example["ner_tags"][word_idx]])
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=False)



Map:   0%|          | 0/3498 [00:00<?, ? examples/s]

#### Define Metrics and Trainer

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)
    true_predictions = [
        [id2label[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    return seqeval.compute(predictions=true_predictions, references=true_labels)


#### Set Training Arguments

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./amharic-ner",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs"
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


#### Training the Model

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()



/tmp/ipython-input-40-3650850417.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Contact Info,Delivery Fee,Loc,Location,Org,Per,Price,Product,Time,Ttl,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,No log,0.157776,"{'precision': 1.0, 'recall': 0.9177631578947368, 'f1': 0.9571183533447685, 'number': 304}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 11}","{'precision': 0.5429036193574623, 'recall': 0.5714897260273972, 'f1': 0.5568300312825859, 'number': 2336}","{'precision': 0.03125, 'recall': 0.016129032258064516, 'f1': 0.02127659574468085, 'number': 124}","{'precision': 0.5548649294764523, 'recall': 0.7170219338894037, 'f1': 0.6256064690026955, 'number': 3237}","{'precision': 0.7952610144390966, 'recall': 0.850356294536817, 'f1': 0.8218863592883107, 'number': 2526}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 57}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 114}","{'precision': 0.6167247386759582, 'recall': 0.589018302828619, 'f1': 0.6025531914893618, 'number': 1202}","{'precision': 0.6805280528052805, 'recall': 0.7637037037037037, 'f1': 0.7197207678883071, 'number': 1350}",0.633574,0.694787,0.662770,0.948760
2,0.340900,0.081312,"{'precision': 0.7827225130890052, 'recall': 0.9835526315789473, 'f1': 0.8717201166180758, 'number': 304}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 11}","{'precision': 0.6696269982238011, 'recall': 0.8069349315068494, 'f1': 0.7318967190836732, 'number': 2336}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 124}","{'precision': 0.7853587115666179, 'recall': 0.8285449490268767, 'f1': 0.8063740228502705, 'number': 3237}","{'precision': 0.9062980030721967, 'recall': 0.934283452098179, 'f1': 0.9200779727095517, 'number': 2526}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 57}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 114}","{'precision': 0.7371916508538899, 'recall': 0.6464226289517471, 'f1': 0.6888297872340425, 'number': 1202}","{'precision': 0.8020499679692504, 'recall': 0.9274074074074075, 'f1': 0.8601855032634834, 'number': 1350}",0.779631,0.821863,0.800190,0.970792
3,0.137500,0.067454,"{'precision': 0.9966666666666667, 'recall': 0.9835526315789473, 'f1': 0.990066225165563, 'number': 304}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 11}","{'precision': 0.7641221374045801, 'recall': 0.8570205479452054, 'f1': 0.8079096045197739, 'number': 2336}","{'precision': 0.7755102040816326, 'recall': 0.6129032258064516, 'f1': 0.6846846846846848, 'number': 124}","{'precision': 0.8326029798422436, 'recall': 0.8804448563484708, 'f1': 0.8558558558558559, 'number': 3237}","{'precision': 0.934368932038835, 'recall': 0.9524940617577197, 'f1': 0.9433444422662223, 'number': 2526}","{'precision': 0.3076923076923077, 'recall': 0.14035087719298245, 'f1': 0.19277108433734938, 'number': 57}","{'precision': 0.2602739726027397, 'recall': 0.16666666666666666, 'f1': 0.2032085561497326, 'number': 114}","{'precision': 0.617737003058104, 'recall': 0.8402662229617305, 'f1': 0.7120197391610856, 'number': 1202}","{'precision': 0.8591455273698264, 'recall': 0.9533333333333334, 'f1': 0.9037921348314607, 'number': 1350}",0.812949,0.884202,0.847080,0.977084
4,0.093600,0.046031,"{'precision': 0.9804560260586319, 'recall': 0.9901315789473685, 'f1': 0.9852700490998363, 'number': 304}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 11}","{'precision': 0.8262243285939969, 'recall': 0.8955479452054794, 'f1': 0.8594905505341002, 'number': 2336}","{'precision': 0.8888888888888888, 'recall': 0.6451612903225806, 'f1': 0.7476635514018692, 'number': 124}","{'precision': 0.8903426791277259, 'recall': 0.882916280506642, 'f1': 0.886613928959206, 'number': 3237}","{'precision': 0.9425911559348332, 'recall': 0.9619952494061758, 'f1': 0.9521943573667712, 'number': 2526}","{'precision': 0.7021276595744681, 'recall': 0.5789473684210527, 'f1': 0.6346153846153846, 'number': 57}","{'precision': 0.3787878787878788, 'recall': 0.21929824561403508, 'f1': 0.27777777777777773, 'number': 114}","{

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 1.0, 'recall': 0.9177631578947368, 'f1': 0.9571183533447685, 'number': 304}" of type <class 'dict'> for key "eval/CONTACT_INFO" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 11}" of type <class 'dict'> for key "eval/DELIVERY_FEE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.5429036193574623, 'recall': 0.5714897260273972, 'f1': 0.5568300312825859, 'number': 2336}" of

TrainOutput(global_step=2190, training_loss=0.1512342082855364, metrics={'train_runtime': 1200.0587, 'train_samples_per_second': 14.574, 'train_steps_per_second': 1.825, 'total_flos': 1142705999001600.0, 'train_loss': 0.1512342082855364, 'epoch': 5.0})

#### Saving the Model

In [ ]:
trainer.save_model("/content/amharic-ner")
tokenizer.save_pretrained("/content/amharic")

('/content/amharic/tokenizer_config.json',
 '/content/amharic/special_tokens_map.json',
 '/content/amharic/sentencepiece.bpe.model',
 '/content/amharic/added_tokens.json',
 '/content/amharic/tokenizer.json')

In [ ]:
import shutil

# Copy model
shutil.copytree("/content/amharic-ner", "/content/drive/MyDrive/amharic-ner")

# Copy tokenizer
shutil.copytree("/content/amharic", "/content/drive/MyDrive/amharic-tokenizer")


'/content/drive/MyDrive/amharic-tokenizer'